In [1]:
%matplotlib inline
%load_ext autoreload

%autoreload 2

In [2]:
import json
import gff3_parsing
import pandas as pd
import glob

In [3]:
def analyze_genome(dataframe, energy_dict, gaps = (4,10), expected_len = 20):
    """
    NEED DOCS
    """
    for index in dataframe.index:
        upstream = dataframe.loc[index,"upstream_sequence"]
        test_string = upstream.replace("T", "U")
        if len(test_string) != expected_len:
            continue
        if test_string.count("A") + test_string.count("U") +\
                                    test_string.count("C") + test_string.count("G") != expected_len:
            continue
            
        energy_list = []
        for gap in range(gaps[0],gaps[1]+1):
             energy_list.append(energy_dict[test_string[-gap - 6: -gap]])

        dataframe.at[index, "energy_binding"] = min(energy_list)
    return dataframe

# Parameters 

In [10]:
sep = "\t"
upstream_len = 20

with open('../Data/energy_files/energyRef_CCUCCU_ensemble_noneConstraint.json', 'r') as infile:
       energy_dict = json.load(infile)

# Creating initial host tsv files

In [11]:
host_ids = [36809,\
           717959,\
           305,\
           1590,\
           435591,\
           90371,\
           1314,\
           357276,\
           657318,\
           1639,\
           1428,\
           470,\
           573,\
           1280,\
           287,\
           562]

for host_id in host_ids:
    host_df, host_genome = gff3_parsing.compile_sequences(["../Data/host_genomes/{}.gff3".format(host_id)],
                                                        ["../Data/host_genomes/{}.fasta".format(host_id)], upstream_len)
    host_df = analyze_genome(host_df, energy_dict)
    host_df.to_csv("../Data/host_genomes/{}.tsv".format(host_id), sep = sep )

In [12]:
###This one is an annoying 2 chromosome case
host_id = 28450

gffs = ["../Data/host_genomes/{}.1.gff3".format(host_id),\
        "../Data/host_genomes/{}.2.gff3".format(host_id)]

fastas = ["../Data/host_genomes/{}.1.fasta".format(host_id),\
          "../Data/host_genomes/{}.2.fasta".format(host_id)]

host_df, host_genome = gff3_parsing.compile_sequences(gffs, fastas, upstream_len)
#combined_id = 28450
host_df = analyze_genome(host_df, energy_dict)
host_df.to_csv("../Data/host_genomes/{}.tsv".format(host_id), sep=sep)

# Creating initial viral tsv files

In [13]:
new_host_ids = host_ids + [28450]

for host_id in new_host_ids:
    for gff_file in glob.glob("../Data/{}_rep_viruses/*.gff".format(host_id)):
        fasta_file = gff_file.replace(".gff", ".fasta")
        tsv_file = gff_file.replace(".gff", ".tsv")

        viral_df, viral_genome = gff3_parsing.compile_sequences([gff_file], [fasta_file], upstream_len)
        viral_df = analyze_genome(viral_df, energy_dict)
        viral_df.to_csv(tsv_file, sep=sep)